In [57]:
import pandas as pd
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras import optimizers
from keras import regularizers
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras.models import load_model
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv('lego_Sets.csv')

In [19]:
df.list_price = df.list_price.astype(float)
df['review_difficulty'] = df['review_difficulty'].astype("category")
df['review_difficulty'] = df['review_difficulty'].cat.reorder_categories(['Very Easy',
                                                               'Easy',
                                                               'Average',
                                                               'Challenging',
                                                               'Very Challenging'],
                                                                        ordered = True)
df['review_difficulty'] = df['review_difficulty'].cat.codes
df.theme_name = df.theme_name.astype("category")
df.ages = df.ages.astype("category")


In [20]:
df1 = df.copy().dropna()
df1 = df1.drop(['prod_desc', 
                'prod_id', 
                'prod_long_desc', 
                'set_name',
                'theme_name',
                'country'], 
                 axis=1)
df2 = pd.get_dummies(df1)

In [21]:
X, Y = df2.drop(['list_price'], axis=1), df2['list_price']

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.3, 
                                                    random_state=361)

In [48]:
def create_model(momentum = 0.2, #git
                activation = 'softplus', #git
                learn_rate = 0.001, #git
                dropout_rate = 0.0, #git
                weight_constraint = 3, #git
                neurons = 100, #git
                init = 'lecun_uniform', #git
                optimizer = 'adagrad'): #git

    # create model
    NN_model = Sequential()
    NN_model.add(Dense(128, 
                   kernel_initializer=init,
                   input_dim = X_train.shape[1], 
                   activation=activation))

    # The Hidden Layers :
    NN_model.add(Dense(neurons, 
                       kernel_initializer=init,
                       activation=activation,
                      kernel_constraint=maxnorm(weight_constraint)))
    NN_model.add(Dense(neurons, 
                       kernel_initializer=init,
                       activation=activation,
                      kernel_constraint=maxnorm(weight_constraint)))
    NN_model.add(Dropout(dropout_rate)) 
    NN_model.add(Dense(neurons, 
                       kernel_initializer=init,
                       activation=activation,
                      kernel_constraint=maxnorm(weight_constraint)))

    # The Output Layer :
    NN_model.add(Dense(1, kernel_initializer=init,
                       activation='linear'))

    
    NN_model.compile(loss='mean_squared_error', 
                  optimizer=optimizer, 
                  metrics=['mean_absolute_error',
                           'mean_squared_error'])
    
    NN_model.fit(X_train, Y_train,
          epochs=300,
          batch_size=64)
    
    return NN_model


model = KerasRegressor(build_fn=create_model, 
                        epochs = 300, 
                        batch_size = 64,
                        verbose = 1)

In [33]:
def grid_search(model = model):
# Use scikit-learn to grid search 
    activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    learn_rate = [0.0001, 0.0005, 0.001, 0.025, 0.005, 0.075, 0.01, 0.025, 0.05, 0.075, 0.1, 0.2]
    dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    weight_constraint=[1, 2, 3, 4, 5]
    neurons = [30, 100, 128, 256]
    init = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    optimizer = [ 'sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']
    ##############################################################
    # grid search epochs, batch size
    epochs = [50, 100] # add 50, 100, 150 etc
    batch_size = [10, 32, 64, 100] # add 5, 10, 20, 40, 60, 80, 100 etc
    param_grid = dict(# epochs=epochs,
#                      batch_size=batch_size,
#                      activation=activation,
#                      momentum=momentum,
#                      learn_rate=learn_rate,
#                      dropout_rate=dropout_rate,
#                      weight_constraint=weight_constraint,
#                      neurons=neurons,
#                      init=init,
#                      optimizer=optimizer
                     )
    ##############################################################
    grid = GridSearchCV(estimator=model, 
                        param_grid=param_grid, 
                        scoring=['r2'],
                        refit = 'r2',
                        n_jobs=1,
                        cv = 2)
    grid_result = grid.fit(X_train, Y_train) 
    ##############################################################
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_r2']
    stds = grid_result.cv_results_['std_test_r2']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
        
grid_search(model)

Epoch 1/100
3566/3566 [==============================] - 4s 1ms/step - loss: 8397.3293 - mean_absolute_error: 27.7558 - mean_squared_error: 8397.3293
Epoch 2/100
3566/3566 [==============================] - 0s 89us/step - loss: 1386.1370 - mean_absolute_error: 18.5350 - mean_squared_error: 1386.1370
Epoch 3/100
3566/3566 [==============================] - 0s 89us/step - loss: 1328.9710 - mean_absolute_error: 18.4944 - mean_squared_error: 1328.9710
Epoch 4/100
3566/3566 [==============================] - 0s 95us/step - loss: 1307.9485 - mean_absolute_error: 18.1887 - mean_squared_error: 1307.9485
Epoch 5/100
3566/3566 [==============================] - 0s 96us/step - loss: 1277.2486 - mean_absolute_error: 18.0432 - mean_squared_error: 1277.2486
Epoch 6/100
3566/3566 [==============================] - 0s 104us/step - loss: 1271.7529 - mean_absolute_error: 18.0320 - mean_squared_error: 1271.7529
Epoch 7/100
3566/3566 [==============================] - 0s 98us/step - loss: 1287.2930 - mean

3566/3566 [==============================] - 0s 91us/step - loss: 1033.8196 - mean_absolute_error: 14.7711 - mean_squared_error: 1033.8196
Epoch 55/100
3566/3566 [==============================] - 0s 88us/step - loss: 1036.0485 - mean_absolute_error: 14.7301 - mean_squared_error: 1036.0485
Epoch 56/100
3566/3566 [==============================] - 0s 87us/step - loss: 1007.6893 - mean_absolute_error: 14.6243 - mean_squared_error: 1007.6893
Epoch 57/100
3566/3566 [==============================] - 0s 86us/step - loss: 1017.6111 - mean_absolute_error: 14.6106 - mean_squared_error: 1017.6111
Epoch 58/100
3566/3566 [==============================] - 0s 88us/step - loss: 1000.5104 - mean_absolute_error: 14.3789 - mean_squared_error: 1000.5104
Epoch 59/100
3566/3566 [==============================] - 0s 102us/step - loss: 1011.6943 - mean_absolute_error: 14.5769 - mean_squared_error: 1011.6943
Epoch 60/100
3566/3566 [==============================] - 0s 90us/step - loss: 990.2280 - mean_absol

3567/3567 [==============================] - 0s 99us/step - loss: 1630.2858 - mean_absolute_error: 19.0357 - mean_squared_error: 1630.2858
Epoch 8/100
3567/3567 [==============================] - 0s 98us/step - loss: 1615.8104 - mean_absolute_error: 18.7352 - mean_squared_error: 1615.8104
Epoch 9/100
3567/3567 [==============================] - 0s 104us/step - loss: 1651.1220 - mean_absolute_error: 18.9415 - mean_squared_error: 1651.1220
Epoch 10/100
3567/3567 [==============================] - 0s 101us/step - loss: 1619.9467 - mean_absolute_error: 18.6026 - mean_squared_error: 1619.9467
Epoch 11/100
3567/3567 [==============================] - 0s 88us/step - loss: 1614.7971 - mean_absolute_error: 18.4964 - mean_squared_error: 1614.7971
Epoch 12/100
3567/3567 [==============================] - 0s 86us/step - loss: 1589.0922 - mean_absolute_error: 18.5116 - mean_squared_error: 1589.0922
Epoch 13/100
3567/3567 [==============================] - 0s 82us/step - loss: 1544.3812 - mean_absol

3567/3567 [==============================] - 0s 84us/step - loss: 1332.3457 - mean_absolute_error: 16.0485 - mean_squared_error: 1332.3457
Epoch 61/100
3567/3567 [==============================] - 0s 88us/step - loss: 1296.1705 - mean_absolute_error: 15.7776 - mean_squared_error: 1296.1705
Epoch 62/100
3567/3567 [==============================] - 0s 84us/step - loss: 1321.3111 - mean_absolute_error: 15.9632 - mean_squared_error: 1321.3111
Epoch 63/100
3567/3567 [==============================] - 0s 87us/step - loss: 1364.6230 - mean_absolute_error: 16.1720 - mean_squared_error: 1364.6230
Epoch 64/100
3567/3567 [==============================] - 0s 90us/step - loss: 1284.1650 - mean_absolute_error: 15.6887 - mean_squared_error: 1284.1650
Epoch 65/100
3567/3567 [==============================] - 0s 86us/step - loss: 1318.4988 - mean_absolute_error: 15.8008 - mean_squared_error: 1318.4988
Epoch 66/100
3567/3567 [==============================] - 0s 83us/step - loss: 1295.1764 - mean_absol

3566/3566 [==============================] - 0s 86us/step - loss: 1243.3653 - mean_absolute_error: 17.1123 - mean_squared_error: 1243.3653
Epoch 14/100
3566/3566 [==============================] - 0s 82us/step - loss: 1189.5169 - mean_absolute_error: 16.8603 - mean_squared_error: 1189.5169
Epoch 15/100
3566/3566 [==============================] - 0s 83us/step - loss: 1207.5743 - mean_absolute_error: 16.7957 - mean_squared_error: 1207.5743
Epoch 16/100
3566/3566 [==============================] - 0s 88us/step - loss: 1173.1801 - mean_absolute_error: 16.7103 - mean_squared_error: 1173.1801
Epoch 17/100
3566/3566 [==============================] - 0s 82us/step - loss: 1177.7437 - mean_absolute_error: 16.5983 - mean_squared_error: 1177.7437
Epoch 18/100
3566/3566 [==============================] - 0s 85us/step - loss: 1153.2472 - mean_absolute_error: 16.1945 - mean_squared_error: 1153.2472
Epoch 19/100
3566/3566 [==============================] - 0s 83us/step - loss: 1141.8828 - mean_absol

...

3566/3566 [==============================] - 0s 95us/step - loss: 972.0186 - mean_absolute_error: 14.3295 - mean_squared_error: 972.0186
Epoch 85/100
3566/3566 [==============================] - 0s 99us/step - loss: 986.7571 - mean_absolute_error: 14.5258 - mean_squared_error: 986.7571
Epoch 86/100
3566/3566 [==============================] - 0s 99us/step - loss: 996.7651 - mean_absolute_error: 14.4452 - mean_squared_error: 996.7651
Epoch 87/100
3566/3566 [==============================] - 0s 98us/step - loss: 987.8468 - mean_absolute_error: 14.4333 - mean_squared_error: 987.8468
Epoch 88/100
3566/3566 [==============================] - 0s 101us/step - loss: 974.6334 - mean_absolute_error: 14.3642 - mean_squared_error: 974.6334
Epoch 89/100
3566/3566 [==============================] - 0s 102us/step - loss: 995.5851 - mean_absolute_error: 14.4745 - mean_squared_error: 995.5851
Epoch 90/100
3566/3566 [==============================] - 0s 98us/step - loss: 977.8180 - mean_absolute_error: 

3567/3567 [==============================] - 0s 104us/step - loss: 1411.8561 - mean_absolute_error: 16.1100 - mean_squared_error: 1411.8561
Epoch 38/100
3567/3567 [==============================] - 0s 102us/step - loss: 1435.3543 - mean_absolute_error: 16.2660 - mean_squared_error: 1435.3543
Epoch 39/100
3567/3567 [==============================] - 0s 102us/step - loss: 1356.8733 - mean_absolute_error: 16.0355 - mean_squared_error: 1356.8733
Epoch 40/100
3567/3567 [==============================] - 0s 99us/step - loss: 1411.6112 - mean_absolute_error: 15.9752 - mean_squared_error: 1411.6112
Epoch 41/100
3567/3567 [==============================] - 0s 99us/step - loss: 1437.4174 - mean_absolute_error: 16.1578 - mean_squared_error: 1437.4174
Epoch 42/100
3567/3567 [==============================] - 0s 103us/step - loss: 1384.5875 - mean_absolute_error: 15.9000 - mean_squared_error: 1384.5875
Epoch 43/100
3567/3567 [==============================] - 0s 102us/step - loss: 1390.1557 - mean_

3567/3567 [==============================] - 0s 101us/step - loss: 1250.3706 - mean_absolute_error: 15.8319 - mean_squared_error: 1250.3706
Epoch 91/100
3567/3567 [==============================] - 0s 102us/step - loss: 1284.9071 - mean_absolute_error: 15.6627 - mean_squared_error: 1284.9071
Epoch 92/100
3567/3567 [==============================] - 0s 100us/step - loss: 1235.2530 - mean_absolute_error: 15.4848 - mean_squared_error: 1235.2530
Epoch 93/100
3567/3567 [==============================] - 0s 98us/step - loss: 1238.7285 - mean_absolute_error: 15.4565 - mean_squared_error: 1238.7285
Epoch 94/100
3567/3567 [==============================] - 0s 101us/step - loss: 1232.0820 - mean_absolute_error: 15.4841 - mean_squared_error: 1232.0820
Epoch 95/100
3567/3567 [==============================] - 0s 100us/step - loss: 1277.9648 - mean_absolute_error: 15.6759 - mean_squared_error: 1277.9648
Epoch 96/100
3567/3567 [==============================] - 0s 99us/step - loss: 1237.3347 - mean_

Epoch 43/100
7133/7133 [==============================] - 1s 102us/step - loss: 1031.2360 - mean_absolute_error: 15.1555 - mean_squared_error: 1031.2360
Epoch 44/100
7133/7133 [==============================] - 1s 102us/step - loss: 1011.5902 - mean_absolute_error: 14.7909 - mean_squared_error: 1011.5902
Epoch 45/100
7133/7133 [==============================] - 1s 103us/step - loss: 1012.2043 - mean_absolute_error: 14.8656 - mean_squared_error: 1012.2043
Epoch 46/100
7133/7133 [==============================] - 1s 99us/step - loss: 971.6783 - mean_absolute_error: 14.7119 - mean_squared_error: 971.6783
Epoch 47/100
7133/7133 [==============================] - 1s 103us/step - loss: 937.9751 - mean_absolute_error: 14.7105 - mean_squared_error: 937.9751
Epoch 48/100
7133/7133 [==============================] - 1s 104us/step - loss: 914.0163 - mean_absolute_error: 14.4035 - mean_squared_error: 914.0163
Epoch 49/100
7133/7133 [==============================] - 1s 102us/step - loss: 956.1765 

7133/7133 [==============================] - 1s 101us/step - loss: 588.7672 - mean_absolute_error: 12.3057 - mean_squared_error: 588.7672
Epoch 98/100
7133/7133 [==============================] - 1s 101us/step - loss: 573.9389 - mean_absolute_error: 12.0189 - mean_squared_error: 573.9389
Epoch 99/100
7133/7133 [==============================] - 1s 104us/step - loss: 600.9008 - mean_absolute_error: 12.3199 - mean_squared_error: 600.9008
Epoch 100/100
7133/7133 [==============================] - 1s 102us/step - loss: 561.4775 - mean_absolute_error: 12.0809 - mean_squared_error: 561.4775
Best: 0.872400 using {'learn_rate': 0.001}
0.862018 (0.015193) with: {'learn_rate': 0.0001}
0.868624 (0.017482) with: {'learn_rate': 0.0005}
0.872400 (0.023779) with: {'learn_rate': 0.001}
0.857560 (0.024492) with: {'learn_rate': 0.025}
0.859090 (0.018423) with: {'learn_rate': 0.005}
0.861182 (0.001985) with: {'learn_rate': 0.075}
0.864107 (0.009582) with: {'learn_rate': 0.01}
0.857595 (0.006051) with: {'

In [49]:
testing = create_model()
neural_pred = list(testing.predict(X_test)[:,0])

Epoch 1/300
7133/7133 [==============================] - 6s 843us/step - loss: 2541.3080 - mean_absolute_error: 22.3142 - mean_squared_error: 2541.3080
Epoch 2/300
7133/7133 [==============================] - 1s 121us/step - loss: 1541.6314 - mean_absolute_error: 19.1776 - mean_squared_error: 1541.6314
Epoch 3/300
7133/7133 [==============================] - 1s 120us/step - loss: 1514.7450 - mean_absolute_error: 18.7651 - mean_squared_error: 1514.7450
Epoch 4/300
7133/7133 [==============================] - 1s 118us/step - loss: 1501.0643 - mean_absolute_error: 18.4647 - mean_squared_error: 1501.0643
Epoch 5/300
7133/7133 [==============================] - 1s 123us/step - loss: 1442.7389 - mean_absolute_error: 18.2846 - mean_squared_error: 1442.7389
Epoch 6/300
7133/7133 [==============================] - 1s 118us/step - loss: 1406.9630 - mean_absolute_error: 18.1350 - mean_squared_error: 1406.9630
Epoch 7/300
7133/7133 [==============================] - 1s 119us/step - loss: 1460.7989

7133/7133 [==============================] - 1s 113us/step - loss: 944.8703 - mean_absolute_error: 14.5666 - mean_squared_error: 944.8703
Epoch 55/300
7133/7133 [==============================] - 1s 112us/step - loss: 973.6221 - mean_absolute_error: 14.7146 - mean_squared_error: 973.6221
Epoch 56/300
7133/7133 [==============================] - 1s 112us/step - loss: 954.2956 - mean_absolute_error: 14.6059 - mean_squared_error: 954.2956
Epoch 57/300
7133/7133 [==============================] - 1s 112us/step - loss: 915.6091 - mean_absolute_error: 14.4770 - mean_squared_error: 915.6091
Epoch 58/300
7133/7133 [==============================] - 1s 112us/step - loss: 921.1796 - mean_absolute_error: 14.5001 - mean_squared_error: 921.1796
Epoch 59/300
7133/7133 [==============================] - 1s 112us/step - loss: 934.6849 - mean_absolute_error: 14.4567 - mean_squared_error: 934.6849
Epoch 60/300
7133/7133 [==============================] - 1s 112us/step - loss: 861.9862 - mean_absolute_er

7133/7133 [==============================] - 1s 113us/step - loss: 564.7708 - mean_absolute_error: 12.1378 - mean_squared_error: 564.7708
Epoch 109/300
7133/7133 [==============================] - 1s 112us/step - loss: 582.7681 - mean_absolute_error: 12.2856 - mean_squared_error: 582.7681
Epoch 110/300
7133/7133 [==============================] - 1s 112us/step - loss: 564.9124 - mean_absolute_error: 12.2003 - mean_squared_error: 564.9124
Epoch 111/300
7133/7133 [==============================] - 1s 113us/step - loss: 585.5594 - mean_absolute_error: 12.2323 - mean_squared_error: 585.5594
Epoch 112/300
7133/7133 [==============================] - 1s 112us/step - loss: 561.0977 - mean_absolute_error: 12.2175 - mean_squared_error: 561.0977
Epoch 113/300
7133/7133 [==============================] - 1s 112us/step - loss: 577.0050 - mean_absolute_error: 12.1821 - mean_squared_error: 577.0050
Epoch 114/300
7133/7133 [==============================] - 1s 109us/step - loss: 602.6609 - mean_absol

Epoch 162/300
7133/7133 [==============================] - 1s 111us/step - loss: 486.7586 - mean_absolute_error: 11.3095 - mean_squared_error: 486.7586
Epoch 163/300
7133/7133 [==============================] - 1s 112us/step - loss: 493.9916 - mean_absolute_error: 11.3519 - mean_squared_error: 493.9916
Epoch 164/300
7133/7133 [==============================] - 1s 112us/step - loss: 501.5477 - mean_absolute_error: 11.3152 - mean_squared_error: 501.5477
Epoch 165/300
7133/7133 [==============================] - 1s 111us/step - loss: 518.6484 - mean_absolute_error: 11.5778 - mean_squared_error: 518.6484
Epoch 166/300
7133/7133 [==============================] - ETA: 0s - loss: 486.9642 - mean_absolute_error: 11.2329 - mean_squared_error: 486.964 - 1s 110us/step - loss: 492.7393 - mean_absolute_error: 11.2826 - mean_squared_error: 492.7393
Epoch 167/300
7133/7133 [==============================] - 1s 118us/step - loss: 463.6021 - mean_absolute_error: 11.0586 - mean_squared_error: 463.6021


7133/7133 [==============================] - 1s 114us/step - loss: 443.1539 - mean_absolute_error: 10.6870 - mean_squared_error: 443.1539
Epoch 216/300
7133/7133 [==============================] - 1s 121us/step - loss: 431.3911 - mean_absolute_error: 10.6208 - mean_squared_error: 431.3911
Epoch 217/300
7133/7133 [==============================] - 1s 112us/step - loss: 431.1774 - mean_absolute_error: 10.6906 - mean_squared_error: 431.1774
Epoch 218/300
7133/7133 [==============================] - 1s 110us/step - loss: 420.7653 - mean_absolute_error: 10.5189 - mean_squared_error: 420.7653
Epoch 219/300
7133/7133 [==============================] - 1s 131us/step - loss: 425.6320 - mean_absolute_error: 10.5647 - mean_squared_error: 425.6320
Epoch 220/300
7133/7133 [==============================] - 1s 122us/step - loss: 416.8091 - mean_absolute_error: 10.5251 - mean_squared_error: 416.8091
Epoch 221/300
7133/7133 [==============================] - 1s 118us/step - loss: 429.5353 - mean_absol

7133/7133 [==============================] - 1s 129us/step - loss: 378.5966 - mean_absolute_error: 10.1198 - mean_squared_error: 378.5966
Epoch 269/300
7133/7133 [==============================] - 1s 118us/step - loss: 363.2303 - mean_absolute_error: 10.0359 - mean_squared_error: 363.2303
Epoch 270/300
7133/7133 [==============================] - 1s 114us/step - loss: 394.0711 - mean_absolute_error: 10.2374 - mean_squared_error: 394.0711
Epoch 271/300
7133/7133 [==============================] - 1s 108us/step - loss: 386.1768 - mean_absolute_error: 10.1727 - mean_squared_error: 386.1768
Epoch 272/300
7133/7133 [==============================] - 1s 107us/step - loss: 377.8881 - mean_absolute_error: 10.1128 - mean_squared_error: 377.8881
Epoch 273/300
7133/7133 [==============================] - 1s 109us/step - loss: 375.6163 - mean_absolute_error: 10.1376 - mean_squared_error: 375.6163
Epoch 274/300
7133/7133 [==============================] - 1s 108us/step - loss: 357.1226 - mean_absol

In [54]:
testing.save("lego_neural.h5")

In [58]:
testing = load_model("lego_neural.h5")
testing

In [59]:
neural_pred = list(testing.predict(X_test)[:,0])

In [60]:
neural_forecast = ({'Actual': Y_test, 'Predicted': neural_pred})

In [61]:
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, neural_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, neural_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, neural_pred)))

Mean Absolute Error: 9.925197036600533
Mean Squared Error: 360.345308911048
Root Mean Squared Error: 18.982763468764183


In [62]:
sklearn.metrics.r2_score(Y_test, neural_pred)

0.9480709202946974